In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import sklearn
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from scipy.fft import fft
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import os

In [ ]:
#Define a function that offsets all repeating WaterYear values
def duplicate_years(df):
    # Create a set to keep track of seen values and a dictionary for offsets
    seen = set()
    offsets = {}
    append = 0

    # Increment duplicates
    for idx, year in enumerate(df['WaterYear']):
        if year in seen:
            # Increment the year using the current offset for this value
            offsets[year] += 1
            df.at[idx, 'WaterYear'] = int(year + offsets[year])
        else:
            # First occurrence, initialize offset
            seen.add(year)
            offsets[year] = 0

    df['WaterYear'] = df['WaterYear'].astype('int')
    return df

In [ ]:
# Load data
data = pd.read_csv('../Data/final_data_temp.csv')

In [ ]:
# Convert WaterYear to date time format and set as index
data["Date"] = pd.to_datetime(data["WaterYear"].astype(str) + "-10-01")
data.set_index("Date", inplace=True)
grouped_data = data.groupby("County")

In [ ]:
# Ensure the forecast directory exists
#output_dir = "../Station_Forecasts"
#os.makedirs(output_dir, exist_ok=True)
new_stations = []

In [ ]:
# Creating rolling mean for time-series data
def create_rolling_mean(data, window_size=3):
    data['rolling_mean'] = data["TotalPrecipitation_inches"].rolling(window=window_size).mean()
    return data

In [ ]:
station_names = []
errors_list_ts = []
# Loop through each unique station and create a model - SARIMAX
for station in list(grouped_data):
    station_name = station[0]
    station_names.append(station_name)
    station = station[1]
    
    # Resample Station data to ensure consistent frequency and fill any missing dates
    station = duplicate_years(station)
    station.index = pd.to_datetime(station["WaterYear"].astype(str) + "-10-01")
    station = create_rolling_mean(station, window_size = 5)

    # Train SARIMAX model with reduced complexity for sparse data
    series = station["TotalPrecipitation_inches"]
    series = series.dropna()

    # Group by date (not datetime) and average
    df_grouped = series.groupby(series.index.date).mean()

    # Convert index back to datetime for further operations (optional)
    df_grouped.index = pd.to_datetime(df_grouped.index)

    # Print the result
    print(df_grouped.to_string())

    series = df_grouped

    # Split data into train and test sets
    split_index = int(len(series) * 0.8)
    train = series[:split_index]
    test = series[split_index:]

    model = SARIMAX(
        train,
        order=(1, 1, 1),
        seasonal_order=(0, 1, 1, 12),
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    model_fit = model.fit(disp=False)
    # Forecast for the test period
    forecast = model_fit.forecast(steps=len(test))

    # Convert forecast and test data to DataFrames for comparison
    test_forecast = pd.DataFrame({'Actual': test, 'Predicted': forecast}, index=test.index)

    mae = mean_absolute_error(test, forecast)
    rmse = np.sqrt(mean_squared_error(test, forecast))
    r_sq = r2_score(test, forecast)

    errors_list_ts.append([mae, rmse, r_sq])

    plt.figure(figsize=(10, 6))
    plt.plot(train, label='Train')
    plt.plot(test, label='Test', color='blue')
    plt.plot(test_forecast['Predicted'], label='Predicted', color='orange')
    plt.legend()
    plt.show()
    

In [ ]:
new_df_ts = pd.DataFrame(errors_list_ts, columns = ['mae', 'rmse', 'r_sq'])
new_df_ts.index = station_names
new_df_ts['county'] = new_df_ts.index
print(new_df_ts)

In [ ]:
new_df_ts.plot(x='county', y=['mae', 'rmse', 'r_sq'], kind='bar', figsize=(10, 6))

plt.xlabel('Counties')
plt.ylabel('Values')
plt.title('Performance Metrics of Standard Time Series')
plt.legend()

# Show plot
plt.show()

In [ ]:
# Creating lag features for time-series data
def create_lag_features(data, lag_steps=1):
    for i in range(1, lag_steps + 1):
        data[f'lag_{i}'] = data["TotalPrecipitation_inches"].shift(i)
    return data

In [ ]:
# Creating rolling mean for time-series data
def create_rolling_mean(data, window_size=3):
    data['rolling_mean'] = data["TotalPrecipitation_inches"].rolling(window=window_size).mean()
    return data

In [ ]:
# Applying Fourier transformation for capturing seasonality
def apply_fourier_transform(data):
    values = data['TotalPrecipitation_inches'].values
    fourier_transform = fft(values)
    data['fourier_transform'] = np.abs(fourier_transform)
    return data

In [ ]:
def forecast_future(data, model, steps=10):
    """
    Forecast future values using the XGBoost model and lagged features. (Imported from model_RF.py)
    """
    future_forecast = []
    # Start with the last row of data
    last_known = station['rolling_mean'].tail(5)

    for _ in range(steps):
        input_data = pd.DataFrame([last_known], columns=['rolling_mean'])
        next_pred = model.predict(input_data)[0]
        future_forecast.append(next_pred)

        # Update the input with the new prediction (rolling window)
        last_known = np.roll(last_known, -1)
        last_known[-1] = next_pred  # Add the predicted value to the input

    return future_forecast

In [ ]:
errors_list = []
models_list = []
stations_list = []
stations_names = []
for station in list(grouped_data):
    station_maes = []
    station_rmses = []

    station_name = station[0]
    stations_names.append(station_name)
    station = station[1]
    # Resample Station data to ensure consistent frequency and fill any missing dates
    station = duplicate_years(station)
    station = station.dropna()
    station.index = pd.to_datetime(station["WaterYear"].astype(str) + "-10-01")
    '''
    try:
        station = station.resample("YS-OCT").asfreq()
    except:
        print(station.to_string())
    '''

    # Applying lag feature creation to the dataset
    #station = create_lag_features(station, lag_steps = 5)
    # Applying rolling mean to the dataset
    station = create_rolling_mean(station, window_size = 5)
    #station = apply_fourier_transform(station)

    #X1 = station[['lag_1', 'lag_2', 'lag_3']]
    X2 = station['rolling_mean'] #We picked the rolling mean because precipitation data is susceptible to unnecessary trends (record high/low rainfall years)
    #X3 = station['fourier_transform']
    y = station['TotalPrecipitation_inches']

    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, shuffle=False)

    param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
    grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    # Training the XGBoost model
    xgb_model = XGBRegressor(**best_params)
    xgb_model.fit(X_train, y_train)

    # Evaluating the XGBoost model on the testing set
    predictions = xgb_model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    r_sq = r2_score(y_test, predictions)

    errors_list.append([mae, rmse, r_sq])

    stations_list.append(station)
    models_list.append(xgb_model)

    #plt.plot(y_train.values, label = 'Training', color = 'green', alpha = 0.8)

    # Plot actual values
    plt.plot(y_test.values, label='Actual', color='blue', alpha=0.8)

    # Plot predicted values
    plt.plot(predictions, label='Prediction', color='orange', alpha=0.8)

    plt.xlabel('Time')
    plt.ylabel('Rainfall')
    plt.title('Rainfall Predictions: {} County'.format(station_name))
    plt.legend()
    plt.show()

In [ ]:
'''
i = 0
for station in stations_list:
    # Generate forecasts for the next 80 years
    forecast_years = 80
    future_forecast = forecast_future(station, models_list[i], steps=forecast_years)

    # Create future dates
    future_dates = pd.date_range(
        start=station.index[-1] + pd.DateOffset(years=1),
        periods=forecast_years,
        freq="AS-OCT"
    )

    # Combine into a DataFrame
    forecast_df = pd.DataFrame({
        "Date": future_dates,
        "Predicted_Precipitation": future_forecast
    }).set_index("Date")

    # Define output file path
    output_path = f"../County_Forecasts/{stations_names[i]}_XGBforecast.txt"

    # Save the forecast to a text file
    with open(output_path, "w") as f:
        f.write(f"Precipitation Forecast for {stations_names[i]} County:\n")
        for date, value in forecast_df["Predicted_Precipitation"].items():
            f.write(f"{date.strftime('%Y-%m-%d')}: {value:.2f} inches\n")

    print(f"Forecast for {stations_names[i]} County saved to {output_path}")
    i += 1
'''

The discrepancies in the length of the XGBoost temperature indices stem from the length of the county data.

In [ ]:
new_df = pd.DataFrame(errors_list, columns = ['mae', 'rmse', 'r_sq'])
new_df.index = station_names
new_df['county'] = new_df.index
print(new_df)

In [ ]:
new_df.plot(x='county', y=['mae', 'rmse', 'r_sq'], kind='bar', figsize=(10, 6))

plt.xlabel('Counties')
plt.ylabel('Values')
plt.title('Performance Metrics of XGBoost')
plt.legend()

# Show plot
plt.show()